# Home Credit

## Step 1: Preprocessing

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, Imputer, MinMaxScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import make_pipeline

In [2]:
# Import dataset
df = pd.read_csv('application_train.csv')
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 300)
display(df.head(), df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(41), object(16)
memory usage: 286.2+ MB


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

None

In [3]:
# ID Duplicates
df['SK_ID_CURR'].value_counts().max()

1

In [4]:
bad_rate = df['TARGET'].value_counts(normalize=True)
bad_rate

0    0.919271
1    0.080729
Name: TARGET, dtype: float64

In [5]:
# Variables type float that should be integer
list_float_to_integer = ['DAYS_REGISTRATION','OWN_CAR_AGE','CNT_FAM_MEMBERS',
    'OBS_30_CNT_SOCIAL_CIRCLE','DEF_30_CNT_SOCIAL_CIRCLE','OBS_60_CNT_SOCIAL_CIRCLE',
    'DEF_60_CNT_SOCIAL_CIRCLE','DAYS_LAST_PHONE_CHANGE','AMT_REQ_CREDIT_BUREAU_HOUR',
    'AMT_REQ_CREDIT_BUREAU_DAY','AMT_REQ_CREDIT_BUREAU_WEEK','AMT_REQ_CREDIT_BUREAU_MON',
    'AMT_REQ_CREDIT_BUREAU_QRT','AMT_REQ_CREDIT_BUREAU_YEAR']

# Categorical variables
list_cat = ['NAME_CONTRACT_TYPE','CODE_GENDER','FLAG_OWN_CAR',
            'FLAG_OWN_REALTY','NAME_TYPE_SUITE',
            'NAME_INCOME_TYPE','NAME_EDUCATION_TYPE',
            'NAME_FAMILY_STATUS','NAME_HOUSING_TYPE',
            'OCCUPATION_TYPE',
            'WEEKDAY_APPR_PROCESS_START','ORGANIZATION_TYPE',
            'FONDKAPREMONT_MODE','HOUSETYPE_MODE','WALLSMATERIAL_MODE',
            'EMERGENCYSTATE_MODE'] 

list_dum = ['FLAG_MOBIL','FLAG_EMP_PHONE','FLAG_WORK_PHONE','FLAG_CONT_MOBILE','FLAG_PHONE',
            'FLAG_EMAIL','REG_REGION_NOT_LIVE_REGION','REG_REGION_NOT_WORK_REGION',
            'LIVE_REGION_NOT_WORK_REGION','REG_CITY_NOT_LIVE_CITY','REG_CITY_NOT_WORK_CITY',
            'LIVE_CITY_NOT_WORK_CITY','FLAG_DOCUMENT_2','FLAG_DOCUMENT_3','FLAG_DOCUMENT_4',
            'FLAG_DOCUMENT_5','FLAG_DOCUMENT_6','FLAG_DOCUMENT_7','FLAG_DOCUMENT_8',
            'FLAG_DOCUMENT_9','FLAG_DOCUMENT_10','FLAG_DOCUMENT_11','FLAG_DOCUMENT_12',
            'FLAG_DOCUMENT_13','FLAG_DOCUMENT_14','FLAG_DOCUMENT_15','FLAG_DOCUMENT_16',
            'FLAG_DOCUMENT_17','FLAG_DOCUMENT_18','FLAG_DOCUMENT_19','FLAG_DOCUMENT_20',
            'FLAG_DOCUMENT_21']

# Numerical variables
list_num = ['CNT_CHILDREN','AMT_INCOME_TOTAL','AMT_CREDIT','AMT_ANNUITY','AMT_GOODS_PRICE',
            'REGION_POPULATION_RELATIVE','DAYS_BIRTH','DAYS_EMPLOYED','DAYS_REGISTRATION',
            'DAYS_ID_PUBLISH','OWN_CAR_AGE','CNT_FAM_MEMBERS','REGION_RATING_CLIENT',
            'REGION_RATING_CLIENT_W_CITY','HOUR_APPR_PROCESS_START',
            'EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','APARTMENTS_AVG','BASEMENTAREA_AVG',
            'YEARS_BEGINEXPLUATATION_AVG','YEARS_BUILD_AVG','COMMONAREA_AVG','ELEVATORS_AVG',
            'ENTRANCES_AVG','FLOORSMAX_AVG','FLOORSMIN_AVG','LANDAREA_AVG',
            'LIVINGAPARTMENTS_AVG','LIVINGAREA_AVG','NONLIVINGAPARTMENTS_AVG',
            'NONLIVINGAREA_AVG','APARTMENTS_MODE','BASEMENTAREA_MODE',
            'YEARS_BEGINEXPLUATATION_MODE','YEARS_BUILD_MODE','COMMONAREA_MODE',
            'ELEVATORS_MODE','ENTRANCES_MODE','FLOORSMAX_MODE','FLOORSMIN_MODE',
            'LANDAREA_MODE','LIVINGAPARTMENTS_MODE','LIVINGAREA_MODE',
            'NONLIVINGAPARTMENTS_MODE','NONLIVINGAREA_MODE','APARTMENTS_MEDI',
            'BASEMENTAREA_MEDI','YEARS_BEGINEXPLUATATION_MEDI','YEARS_BUILD_MEDI',
            'COMMONAREA_MEDI','ELEVATORS_MEDI','ENTRANCES_MEDI','FLOORSMAX_MEDI',
            'FLOORSMIN_MEDI','LANDAREA_MEDI','LIVINGAPARTMENTS_MEDI','LIVINGAREA_MEDI',
            'NONLIVINGAPARTMENTS_MEDI','NONLIVINGAREA_MEDI','TOTALAREA_MODE',
            'OBS_30_CNT_SOCIAL_CIRCLE','DEF_30_CNT_SOCIAL_CIRCLE','OBS_60_CNT_SOCIAL_CIRCLE',
            'DEF_60_CNT_SOCIAL_CIRCLE','DAYS_LAST_PHONE_CHANGE','AMT_REQ_CREDIT_BUREAU_HOUR',
            'AMT_REQ_CREDIT_BUREAU_DAY','AMT_REQ_CREDIT_BUREAU_WEEK','AMT_REQ_CREDIT_BUREAU_MON',
            'AMT_REQ_CREDIT_BUREAU_QRT','AMT_REQ_CREDIT_BUREAU_YEAR']

In [6]:
# CATEGORICAL VARIABLES
for c in list_cat:
    aux=df[['TARGET',c,'SK_ID_CURR']].fillna('nulls').groupby(by=c).agg({'TARGET':sum,'SK_ID_CURR':'count'}).rename(columns={'TARGET':'bad','SK_ID_CURR':'total'})
    aux['bad_rate']=aux['bad']/aux['total']
    aux['total_%']=aux['total']/aux['total'].sum()
    aux['bad_%']=aux['bad']/aux['bad'].sum()
    aux=aux.reindex(columns=['total','total_%','bad','bad_%','bad_rate'])
    display(aux)
    
# Next steps:
# CODE_GENDER: 4 XNA (impute most frequent: F)
# NAME_TYPE_SUITE: 1292 nulls (impute most frequent: Unaccompanied)
# OCCUPATION_TYPE: 31% nulls (new category NULL)
# ORGANIZATION_TYPE: 18% XNA (new category NULL)
# FONDKAPREMONT_MODE: 68% nulls (new category NULL)
# HOUSETYPE_MODE: 50% nulls (new category NULL)
# WALLSMATERIAL_MODE: 50% nulls (new category NULL)
# EMERGENCYSTATE_MODE: 47% nulls (new category NULL)

,total,total_%,bad,bad_%,bad_rate
NAME_CONTRACT_TYPE,,,,,
Cash loans,278232,0.904787,23221,0.935388,0.083459
Revolving loans,29279,0.095213,1604,0.064612,0.054783


,total,total_%,bad,bad_%,bad_rate
CODE_GENDER,,,,,
F,202448,0.658344,14170,0.570796,0.069993
M,105059,0.341643,10655,0.429204,0.101419
XNA,4,0.000013,0,0.000000,0.000000


,total,total_%,bad,bad_%,bad_rate
FLAG_OWN_CAR,,,,,
N,202924,0.659892,17249,0.694824,0.085002
Y,104587,0.340108,7576,0.305176,0.072437


,total,total_%,bad,bad_%,bad_rate
FLAG_OWN_REALTY,,,,,
N,94199,0.306327,7842,0.315891,0.083249
Y,213312,0.693673,16983,0.684109,0.079616


,total,total_%,bad,bad_%,bad_rate
NAME_TYPE_SUITE,,,,,
Children,3267,0.010624,241,0.009708,0.073768
Family,40149,0.130561,3009,0.121208,0.074946
Group of people,271,0.000881,23,0.000926,0.084871
Other_A,866,0.002816,76,0.003061,0.087760
Other_B,1770,0.005756,174,0.007009,0.098305
"Spouse, partner",11370,0.036974,895,0.036052,0.078716
Unaccompanied,248526,0.808186,20337,0.819215,0.081830
nulls,1292,0.004201,70,0.002820,0.054180


...

,total,total_%,bad,bad_%,bad_rate
ORGANIZATION_TYPE,,,,,
Advertising,429,0.001395,35,0.001410,0.081585
Agriculture,2454,0.007980,257,0.010352,0.104727
Bank,2507,0.008153,130,0.005237,0.051855
Business Entity Type 1,5984,0.019459,487,0.019617,0.081384
Business Entity Type 2,10553,0.034317,900,0.036254,0.085284
Business Entity Type 3,67992,0.221104,6323,0.254703,0.092996
Cleaning,260,0.000845,29,0.001168,0.111538
Construction,6721,0.021856,785,0.031621,0.116798
Culture,379,0.001232,21,0.000846,0.055409


,total,total_%,bad,bad_%,bad_rate
FONDKAPREMONT_MODE,,,,,
not specified,5687,0.018494,429,0.017281,0.075435
nulls,210295,0.683862,18125,0.730111,0.086188
org spec account,5619,0.018273,327,0.013172,0.058195
reg oper account,73830,0.240089,5152,0.207533,0.069782
reg oper spec account,12080,0.039283,792,0.031903,0.065563


,total,total_%,bad,bad_%,bad_rate
HOUSETYPE_MODE,,,,,
block of flats,150503,0.489423,10450,0.420947,0.069434
nulls,154297,0.501761,14120,0.568781,0.091512
specific housing,1499,0.004875,152,0.006123,0.101401
terraced house,1212,0.003941,103,0.004149,0.084983


,total,total_%,bad,bad_%,bad_rate
WALLSMATERIAL_MODE,,,,,
Block,9253,0.030090,650,0.026183,0.070247
Mixed,2296,0.007466,173,0.006969,0.075348
Monolithic,1779,0.005785,84,0.003384,0.047218
Others,1625,0.005284,135,0.005438,0.083077
Panel,66040,0.214757,4192,0.168862,0.063477
"Stone, brick",64815,0.210773,4800,0.193353,0.074057
Wooden,5362,0.017437,520,0.020947,0.096979
nulls,156341,0.508408,14271,0.574864,0.091281


,total,total_%,bad,bad_%,bad_rate
EMERGENCYSTATE_MODE,,,,,
No,159428,0.518446,11104,0.447291,0.069649
Yes,2328,0.007570,223,0.008983,0.095790
nulls,145755,0.473983,13498,0.543726,0.092607


In [7]:
# DUMMIES

df_dum=pd.DataFrame()
for c in list_dum:
    aux=pd.DataFrame(df.loc[:,c].fillna('nulls').value_counts(normalize=True))
    df_dum=pd.concat([df_dum,aux],axis=1)

display(df_dum)

# next steps: N/A

,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21
0,0.000003,0.180111,0.800632,0.001867,0.718934,0.94328,0.984856,0.949231,0.959341,0.921827,0.769546,0.820445,0.999958,0.289977,0.999919,0.984885,0.911945,0.999808,0.918624,0.996104,0.999977,0.996088,0.999993,0.996475,0.997064,0.99879,0.990072,0.999733,0.99187,0.999405,0.999493,0.999665
1,0.999997,0.819889,0.199368,0.998133,0.281066,0.05672,0.015144,0.050769,0.040659,0.078173,0.230454,0.179555,0.000042,0.710023,0.000081,0.015115,0.088055,0.000192,0.081376,0.003896,0.000023,0.003912,0.000007,0.003525,0.002936,0.00121,0.009928,0.000267,0.00813,0.000595,0.000507,0.000335


In [8]:
# NUMERICAL VARIABLES
df_num=df[list_num]  
df_num.describe(percentiles=[0.10, 0.25, 0.75, 0.90,0.95,0.999]).applymap(lambda x: "{:.2f}".format(x)) 


,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,307511.00,307511.00,307511.00,307499.00,307233.00,307511.00,307511.00,307511.00,307511.00,307511.00,104582.00,307509.00,307511.00,307511.00,307511.00,134133.00,306851.00,246546.00,151450.00,127568.00,157504.00,103023.00,92646.00,143620.00,152683.00,154491.00,98869.00,124921.00,97312.00,153161.00,93997.00,137829.00,151450.00,127568.00,157504.00,103023.00,92646.00,143620.00,152683.00,154491.00,98869.00,124921.00,97312.00,153161.00,93997.00,137829.00,151450.00,127568.00,157504.00,103023.00,92646.00,143620.00,152683.00,154491.00,98869.00,124921.00,97312.00,153161.00,93997.00,137829.00,159080.00,306490.00,306490.00,306490.00,306490.00,307510.00,265992.00,265992.00,265992.00,265992.00,265992.00,265992.00
mean,0.42,168797.92,599026.00,27108.57,538396.21,0.02,-16037.00,63815.05,-4986.12,-2994.20,12.06,2.15,2.05,2.03,12.06,0.50,0.51,0.51,0.12,0.09,0.98,0.75,0.04,0.08,0.15,0.23,0.23,0.07,0.10,0.11,0.01,0.03,0.11,0.09,0.98,0.76,0.04,0.07,0.15,0.22,0.23,0.06,0.11,0.11,0.01,0.03,0.12,0.09,0.98,0.76,0.04,0.08,0.15,0.23,0.23,0.07,0.10,0.11,0.01,0.03,0.10,1.42,0.14,1.41,0.10,-962.86,0.01,0.01,0.03,0.27,0.27,1.90
std,0.72,237123.15,402490.78,14493.74,369446.46,0.01,4363.99,141275.77,3522.89,1509.45,11.94,0.91,0.51,0.50,3.27,0.21,0.19,0.19,0.11,0.08,0.06,0.11,0.08,0.13,0.10,0.14,0.16,0.08,0.09,0.11,0.05,0.07,0.11,0.08,0.06,0.11,0.07,0.13,0.10,0.14,0.16,0.08,0.10,0.11,0.05,0.07,0.11,0.08,0.06,0.11,0.08,0.13,0.10,0.15,0.16,0.08,0.09,0.11,0.05,0.07,0.11,2.40,0.45,2.38,0.36,826.81,0.08,0.11,0.20,0.92,0.79,1.87
min,0.00,25650.00,45000.00,1615.50,40500.00,0.00,-25229.00,-17912.00,-24672.00,-7197.00,0.00,1.00,1.00,1.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-4292.00,0.00,0.00,0.00,0.00,0.00,0.00
10%,0.00,81000.00,180000.00,11074.50,180000.00,0.01,-22181.00,-4881.00,-9932.00,-4724.00,2.00,1.00,1.00,1.00,8.00,0.21,0.22,0.23,0.02,0.00,0.97,0.63,0.00,0.00,0.03,0.04,0.04,0.00,0.02,0.01,0.00,0.00,0.02,0.00,0.97,0.65,0.00,0.00,0.03,0.04,0.04,0.00,0.02,0.01,0.00,0.00,0.02,0.00,0.97,0.64,0.00,0.00,0.03,0.04,0.04,0.00,0.02,0.01,0.00,0.00,0.01,0.00,0.00,0.00,0.00,-2158.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,0.00,112500.00,270000.00,16524.00,238500.00,0.01,-19682.00,-2760.00,-7479.50,-4299.00,5.00,2.00,2.00,2.00,10.00,0.33,0.39,0.37,0.06,0.04,0.98,0.69,0.01,0.00,0.07,0.17,0.08,0.02,0.05,0.05,0.00,0.00,0.05,0.04,0.98,0.70,0.01,0.00,0.07,0.17,0.08,0.02,0.05,0.04,0.00,0.00,0.06,0.04,0.98,0.69,0.01,0.00,0.07,0.17,0.08,0.02,0.05,0.05,0.00,0.00

In [9]:
# 1. MISSING VALUES
df2 = df

# Replace XNA
df2 = df2.applymap(lambda x: np.nan if x=='XNA' else x)

# Replace missing value in Days Employed
df2['DAYS_EMPLOYED'] = df2['DAYS_EMPLOYED'].apply(lambda x: np.nan if x==365243 else x)

In [10]:
# 2. NEW ATTRIBUTES
df3=df2

# Flag Employed
df3['FLAG_EMPLOYED']=df3.apply(lambda x: 0 if np.isnan(x['DAYS_EMPLOYED']) else 1, axis=1)

# Flag Weekend
df3['FLAG_WEEKEND']=df3['WEEKDAY_APPR_PROCESS_START'].apply(lambda x: 1 if ( x=='SUNDAY' or x=='SATURDAY') else 0)

# Score
df3['EXT_SOURCE_MEAN']=df3.apply(lambda x: np.nanmean([x['EXT_SOURCE_1'],x['EXT_SOURCE_2'],x['EXT_SOURCE_3']]), axis=1) 

# Delete Outlier AMT_INCOME_TOTAL
df3 = df3[df3['AMT_INCOME_TOTAL']<110000000]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: Mean of empty slice
  # This is added back by InteractiveShellApp.init_path()


In [11]:
# Lists to be used in classes
#1.
list_cat_mf = ['CODE_GENDER','NAME_TYPE_SUITE'] 
#2.
list_cat_null = ['OCCUPATION_TYPE','ORGANIZATION_TYPE',
                 'FONDKAPREMONT_MODE','HOUSETYPE_MODE',
                 'WALLSMATERIAL_MODE','EMERGENCYSTATE_MODE'] 
#3.
list_cat_encoder = ['NAME_EDUCATION_TYPE']
name_variable_net='NAME_EDUCATION_TYPE'
dic_labels_net={'Lower secondary':1,'Secondary / secondary special':2,
                'Incomplete higher':3,'Higher education':4,
                'Academic degree':5}
#4.
list_drop = ['WEEKDAY_APPR_PROCESS_START','EXT_SOURCE_1',
            'EXT_SOURCE_2','EXT_SOURCE_3', 'FLAG_MOBIL'] 
#5.
list_cat_dummies = ['NAME_CONTRACT_TYPE','CODE_GENDER','FLAG_OWN_CAR',
            'FLAG_OWN_REALTY','NAME_TYPE_SUITE','NAME_INCOME_TYPE',
            'NAME_FAMILY_STATUS','NAME_HOUSING_TYPE',
            'OCCUPATION_TYPE','ORGANIZATION_TYPE',
            'FONDKAPREMONT_MODE','HOUSETYPE_MODE','WALLSMATERIAL_MODE',
            'EMERGENCYSTATE_MODE'] 

#6.
list_num_outliers = ['AMT_INCOME_TOTAL']
# 'AMT_INCOME_TOTAL': 117.000.000 outlier

#7.
list_num_zero = ['DAYS_EMPLOYED']

#8.
list_num_max = ['OWN_CAR_AGE']

In [12]:
#3. TRAIN TEST SPLIT
y = df3['TARGET']
X = df3.drop(columns=['SK_ID_CURR','TARGET'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123, stratify=y)

In [13]:
bad_rate_train = y_train.value_counts(normalize=True)
bad_rate_test = y_test.value_counts(normalize=True)
display(bad_rate_train,bad_rate_test)

0    0.919275
1    0.080725
Name: TARGET, dtype: float64

0    0.919271
1    0.080729
Name: TARGET, dtype: float64

In [14]:
#4. IMPUTE

# 4.1 

class ImputerMostFrequentCat(BaseEstimator, TransformerMixin):
    """Imputes most frequent value in categorical variables"""
    def __init__(self,parameter):
        self.a = parameter
        self.b = []
        
    def fit(self,X,*_):
        for i in self.a:
            self.b.append(X[i].mode()[0])
        return self
    
    def transform(self,Z,*_):
        L = Z.copy()
        h=0
        for i in self.a:
            L[i]=L[i].apply(lambda x: self.b[h] if pd.isnull(x) else x)
            h=h+1
        return L

In [15]:
# 4.2 
class ImputerNewNullCat(BaseEstimator, TransformerMixin):
    """Creates a new category called null"""
    def __init__(self,parameter):
        self.a = parameter
        
    def fit(self,X,*_):
        return self
    
    def transform(self,Z,*_):
        L = Z.copy()
        for i in self.a:
            L[i]=L[i].apply(lambda x: 'null_cat' if pd.isnull(x) else x)
        return L

In [16]:
# 4.3 
class LabelEncoderNET(BaseEstimator, TransformerMixin):
    """Encodes labels from Name_Education_Type"""
    def __init__(self,name_variable, dic_labels):
        self.a = name_variable
        self.b = dic_labels
        
    def fit(self,X,*_):
        return self
    
    def transform(self,Z,*_):
        L = Z.copy()
        L[self.a]=Z[self.a].apply(lambda x: self.b[x])
        return L

In [17]:
# 4.4 
class DropVariables(BaseEstimator, TransformerMixin):
    """Drops variables"""
    def __init__(self,list_variables):
        self.a = list_variables
        
    def fit(self,X,*_):
        return self
        
    def transform(self,Z,*_):
        L = Z.copy()
        return L.drop(columns=self.a)

In [18]:
# 4.5 
class Dummies(BaseEstimator, TransformerMixin):
    """Gets dummies without dropping the last column"""
    def __init__(self,list_variables):
        self.a = list_variables
        self.b = {}
        
    def fit(self,X,*_):
        for i in self.a:
            self.b.update({i: list(pd.get_dummies(X[i]).columns)})
        return self
        
    def transform(self,Z,*_):
        L = Z.copy()
        for h in self.a:
            for g in self.b[h]:
                L[h+'_'+g]=L[h].apply(lambda x: 1 if x==g else 0)
        return L.drop(columns=self.a)

In [19]:
# 4.6 
class ImputerZero(BaseEstimator, TransformerMixin):
    """Imputes zero"""
    def __init__(self,parameter):
        self.a = parameter
        
    def fit(self,X,*_):
        return self
    
    def transform(self,Z,*_):
        L = Z.copy()
        for i in self.a:
            L[i]=L[i].apply(lambda x: 0 if np.isnan(x) else x)
        return L

In [20]:
# 4.7 
class ImputerMax(BaseEstimator, TransformerMixin):
    """Imputes max"""
    def __init__(self,parameter):
        self.a = parameter
        self.b = {}
        
    def fit(self,X,*_):
        for i in self.a:
            self.b.update({i: X[i].max()})
        return self
    
    def transform(self,Z,*_):
        L = Z.copy()
        for i in self.a:
            L[i]=L[i].apply(lambda x: self.b[i] if np.isnan(x) else x)
        return L

In [21]:
# 4.8 
class ImputerMean(BaseEstimator, TransformerMixin):
    """Imputes mean"""
    def __init__(self):
        self.a = []
        self.b = {}
        
    def fit(self,X,*_):
        for i in X.columns:
            if ((X[i].dtype=='float64')|(X[i].dtype=='int64')):
                self.a.append(i)
        for i in self.a:
            self.b.update({i: X[i].mean()})
        return self
    
    def transform(self,Z,*_):
        L = Z.copy()
        for i in self.a:
            L[i]=L[i].apply(lambda x: self.b[i] if np.isnan(x) else x)
        return L

In [22]:
# 4.9 
class MinMaxScalerNeg(BaseEstimator, TransformerMixin):
    """MinMaxScaler Negative"""
    def __init__(self):
        self.a = {}
        self.b = {}
        
    def fit(self,X,*_):
        for i in X.columns:
            if X[i].min()>=0:
                self.a.update({i: X[i].min()})
            elif X[i].min()<0:
                self.a.update({i: X[i].min()*1.1})  
        for i in X.columns:
            if X[i].max()>=0:
                self.b.update({i: X[i].max()})
            elif X[i].max()<0:
                self.b.update({i: X[i].max()*0.9}) 
        return self
    
    def transform(self,Z,*_):
        L = Z.copy()
        for i in L.columns:
            L[i]=L[i].apply(lambda x: (x-self.a[i])/(self.b[i]-self.a[i]))
        return L

In [23]:
# Pipeline
preprocessing_pipe = make_pipeline(ImputerMostFrequentCat(list_cat_mf),
                                   ImputerNewNullCat(list_cat_null),
                                   LabelEncoderNET(name_variable_net,dic_labels_net),
                                   DropVariables(list_drop),
                                   ImputerZero(list_num_zero),
                                   ImputerMax(list_num_max),
                                   ImputerMean(),
                                   Dummies(list_cat_dummies),
                                   MinMaxScalerNeg())

## STEP 2: Feature selection

In [24]:
from sklearn.feature_selection import VarianceThreshold, SelectKBest, SelectPercentile, chi2, f_classif, f_regression

In [25]:
# 5.1 
class FeatureSelectionExecuter(BaseEstimator,TransformerMixin):
    """Feature Selection Executer"""
    def __init__(self,instclass,withy):
        self.a=instclass
        self.b=withy
        
    def fit(self,X,y,*_):
        if self.b == "No":
            self.a.fit(X)
        else:
            self.a.fit(X,y)
        return self
    
    def transform(self,Z,*_): 
        return pd.DataFrame(self.a.transform(Z), columns=Z.columns[self.a.get_support()])

In [26]:
# FEATURE SELECTION:
feature_selection_pipeline = make_pipeline(FeatureSelectionExecuter(VarianceThreshold(threshold=0.00001),"No"),
                                           FeatureSelectionExecuter(SelectPercentile(chi2,percentile=80),"Yes"))

## Executing pipelines

In [27]:
# PREPROCESSING:
X_train_p = preprocessing_pipe.fit_transform(X_train)
X_test_p = preprocessing_pipe.transform(X_test)

In [28]:
# FEATURE SELECTION
X_train_fs = feature_selection_pipeline.fit_transform(X_train_p,y_train)
X_test_fs = feature_selection_pipeline.transform(X_test_p)

In [29]:
# check dropped variables:
dropvar=[]
for i in X_train_p.columns:
    if i in X_train_fs.columns:
        dropvar=dropvar
    else:
        dropvar.append(i)
display(X_train_p.columns.nunique(), len(dropvar), dropvar)

236

49

['AMT_INCOME_TOTAL',
 'FLAG_CONT_MOBILE',
 'YEARS_BEGINEXPLUATATION_AVG',
 'YEARS_BUILD_AVG',
 'LANDAREA_AVG',
 'NONLIVINGAPARTMENTS_AVG',
 'YEARS_BEGINEXPLUATATION_MODE',
 'YEARS_BUILD_MODE',
 'COMMONAREA_MODE',
 'LANDAREA_MODE',
 'NONLIVINGAPARTMENTS_MODE',
 'YEARS_BEGINEXPLUATATION_MEDI',
 'YEARS_BUILD_MEDI',
 'NONLIVINGAPARTMENTS_MEDI',
 'OBS_30_CNT_SOCIAL_CIRCLE',
 'OBS_60_CNT_SOCIAL_CIRCLE',
 'FLAG_DOCUMENT_5',
 'FLAG_DOCUMENT_7',
 'FLAG_DOCUMENT_10',
 'FLAG_DOCUMENT_12',
 'FLAG_DOCUMENT_20',
 'AMT_REQ_CREDIT_BUREAU_HOUR',
 'AMT_REQ_CREDIT_BUREAU_DAY',
 'AMT_REQ_CREDIT_BUREAU_WEEK',
 'AMT_REQ_CREDIT_BUREAU_QRT',
 'NAME_TYPE_SUITE_Spouse, partner',
 'NAME_FAMILY_STATUS_Unknown',
 'NAME_HOUSING_TYPE_Co-op apartment',
 'OCCUPATION_TYPE_Realty agents',
 'ORGANIZATION_TYPE_Advertising',
 'ORGANIZATION_TYPE_Business Entity Type 1',
 'ORGANIZATION_TYPE_Cleaning',
 'ORGANIZATION_TYPE_Emergency',
 'ORGANIZATION_TYPE_Housing',
 'ORGANIZATION_TYPE_Industry: type 2',
 'ORGANIZATION_TYPE_Indu

In [30]:
# Export dataset
X_train_fs.to_csv('X_train_2.csv')
X_test_fs.to_csv('X_test_2.csv')
y_train.to_csv('y_train_2.csv')
y_test.to_csv('y_test_2.csv')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """
